In [1]:
import numpy as np

import tensorflow as tf
import tensorflow.contrib.layers as layers

"""
Meant as a pure neural network library

RL actions such as argmaxes etc are all done elsewhere
"""

def linear(input_tensor, nodes, scope, use_relu=True, layer_norm=False):  
    """
    
    Weights are initialized using Xavier-Glorot, biases at zero (these are the defaults )
    
    """
    
    layer = layers.fully_connected(
        input_tensor, 
        num_outputs=nodes, 
        activation_fn=None,
        scope='{}_layer'.format(scope))
            
    if layer_norm:
        layer = layers.layer_norm(
            out, 
            center=True, 
            scale=True,
            scope='{}_layer_norm'.format(scope))
        
    if use_relu:
        layer = tf.nn.relu(
            layer, 
            name='{}_activation'.format(scope))
        
    return layer

def feed_forward(hiddens, observation, scope, layer_norm=False):
    """
    
    """
    with tf.variable_scope(scope, reuse=False):
        layer = observation
        
        for num, hidden in enumerate(hiddens):
            layer = linear(layer, hidden, 'hidden_{}'.format(num), use_relu=True)
                      
        return layer


/Users/adam/anaconda3/envs/energy_py/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)
/Users/adam/anaconda3/envs/energy_py/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Use the retry module or similar alternatives.


Notes on the advantage
https://github.com/gokhanettin/dddqn-tf/blob/master/dddqn.py
```
# See Eq. (9) in https://arxiv.org/pdf/1511.06581.pdf
q_values = value + tf.sub(advantage,
                   tf.reduce_mean(advantage, reduction_indices=1, keep_dims=True))
```

https://github.com/devsisters/DQN-tensorflow/blob/master/dqn/agent.py
https://github.com/carpedm20/deep-rl-tensorflow

But how is target made for this network???
```
        self.q = self.value + (self.advantage - 
          tf.reduce_mean(self.advantage, reduction_indices=1, keep_dims=True))
```

https://github.com/reinforceio/tensorforce/blob/master/tensorforce/models/q_naf_model.py
something totally different
possible that the above methods do this computation elsewhere

In [ ]:

def dueling_output_layer(in_layer, hiddens_value, hiddens_advantage, num_actions):
    """
    Inspired by deep-rl-tensorflow
    
    Not sure if this is doing everything that DeepMind do in the paper
    
    returns
        q_values (tensor) shape=(batch_size, num_actions)
    """
    
    layer = in_layer 
    for num, hidden in enumerate(hiddens_value):
        layer = linear(layer, hidden, 'value_hidden_{}'.format(num), use_relu=True)
    value = linear(layer, 1, 'value_out_{}'.format(num), use_relu=False)
    
    layer = in_layer
    for num, hidden in enumerate(hiddens_advantage):
        layer = linear(layer, hidden, 'advantage_hidden_{}'.format(num), use_relu=True)
    advantage = linear(layer, num_actions, 'advantage_out_{}'.format(num), use_relu=False)
    
    # avarage dueling - apparently (needs checking
    return value + tf.sub(advantage, tf.reduce_mean(advantage, reduction_indices=1, keep_dims=True))
